# Importing Libraries

In [1]:
#basic Libraries
import numpy as np
import pandas as pd
import warnings
import scipy

#Data Visualization
import seaborn as sns
import matplotlib.pyplot as plt

#Libraries set up
warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = (10, 10)
plt.style.use('seaborn')


#NLP
from bs4 import BeautifulSoup 
import re
from nltk.corpus import stopwords
import spacy
from nltk.tokenize import RegexpTokenizer
import nltk

#Machine Learning libraries
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.feature_selection import RFE as RFE
from sklearn.metrics import r2_score

# 1. Loading the dataset

In [2]:
df = pd.read_csv("spotify_dataset.csv", encoding='ISO-8859-1')

#got an error (UnicodeDecodeError: "utf-8") when loading the dataset for the 1st time. 
#The UnicodeDecodeError: ‘utf-8’ codec can’t decode bytes in position 0-1: invalid continuation byte error 
#is a common error that occurs when trying to read a file with Pandas that contains non-UTF-8 encoded characters. 
#UTF-8 is a character encoding standard that’s widely used for text files, but it’s not the only encoding format out there. 
#If you try to read a file with a different encoding format, you may encounter this error.

# 2. Initial exploration

In [60]:
df.head()

,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,...,bpm,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%
0,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",2,2023,7,14,553,147,141381703,43,...,125,B,Major,80,89,83,31,0,8,4
1,LALA,Myke Towers,1,2023,3,23,1474,48,133716286,48,...,92,C#,Major,71,61,74,7,0,10,4
2,vampire,Olivia Rodrigo,1,2023,6,30,1397,113,140003974,94,...,138,F,Major,51,32,53,17,0,31,6
3,Cruel Summer,Taylor Swift,1,2019,8,23,7858,100,800840817,116,...,170,A,Major,55,58,72,11,0,11,15
4,WHERE SHE GOES,Bad Bunny,1,2023,5,18,3133,50,303236322,84,...,144,A,Minor,65,23,80,14,63,11,6


In [153]:
df.shape

(952, 24)

In [154]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 952 entries, 0 to 951
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   track_name            952 non-null    object 
 1   artist(s)_name        952 non-null    object 
 2   artist_count          952 non-null    int64  
 3   released_year         952 non-null    int64  
 4   released_month        952 non-null    int64  
 5   released_day          952 non-null    int64  
 6   in_spotify_playlists  952 non-null    int64  
 7   in_spotify_charts     952 non-null    int64  
 8   streams               952 non-null    int64  
 9   in_apple_playlists    952 non-null    int64  
 10  in_apple_charts       952 non-null    int64  
 11  in_deezer_playlists   952 non-null    int64  
 12  in_deezer_charts      952 non-null    int64  
 13  in_shazam_charts      902 non-null    float64
 14  bpm                   952 non-null    int64  
 15  key                   8

In [3]:
conditions = [    
    (df["released_month"] > 11) | (df["released_month"] <= 3),
    (df["released_month"] == 4) | (df["released_month"] == 5),
    (df["released_month"] >= 6) | (df["released_month"] <= 9),
    (df["released_month"] == 10) | (df["released_month"] == 11)
]

# create a list of the values we want to assign for each condition
values = ['Winter', 'Spring', 'Summer', 'Fall']
    
# create a new column and use np.select to assign values to it using our lists as arguments
df['released_season'] = np.select(conditions, values)

In [156]:
df.head(10)

,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,...,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%,released_season
0,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",2,2023,7,14,553,147,141381703,43,...,B,Major,80,89,83,31,0,8,4,Summer
1,LALA,Myke Towers,1,2023,3,23,1474,48,133716286,48,...,C#,Major,71,61,74,7,0,10,4,Winter
2,vampire,Olivia Rodrigo,1,2023,6,30,1397,113,140003974,94,...,F,Major,51,32,53,17,0,31,6,Summer
3,Cruel Summer,Taylor Swift,1,2019,8,23,7858,100,800840817,116,...,A,Major,55,58,72,11,0,11,15,Summer
4,WHERE SHE GOES,Bad Bunny,1,2023,5,18,3133,50,303236322,84,...,A,Minor,65,23,80,14,63,11,6,Spring
5,Sprinter,"Dave, Central Cee",2,2023,6,1,2186,91,183706234,67,...,C#,Major,92,66,58,19,0,8,24,Summer
6,Ella Baila Sola,"Eslabon Armado, Peso Pluma",2,2023,3,16,3090,50,725980112,34,...,F,Minor,67,83,76,48,0,8,3,Winter
7,Columbia,Quevedo,1,2023,7,7,714,43,58149378,25,...,F,Major,67,26,71,37,0,11,4,Summer
8,fukumean,Gunna,1,2023,5,15,1096,83,95217315,60,...,C#,Minor,85,22,62,12,0,28,9,Spring
9,La Bebe - Remix,"Peso Pluma, Yng Lvcas",2,2023,3,17,2953,44,553634067,49,...,D,Minor,81,56,48,21,0,8,33,Winter


# 3. Data cleaning and imputation

In [4]:
#Dropping columns not needed for this analysis

df.drop(["track_name", "artist(s)_name", "released_year", "released_month", "released_day"], axis=1, inplace=True)

In [245]:
df.shape

(952, 24)

In [246]:
# Checking for null values, we found some missing values in 2 columns.
#"in_shazam_charts" has 50/952 = 5% missing values and "key" has 10% missing values.
df.isnull().sum()

artist_count             0
in_spotify_playlists     0
in_spotify_charts        0
streams                  0
in_apple_playlists       0
in_apple_charts          0
in_deezer_playlists      0
in_deezer_charts         0
in_shazam_charts        50
bpm                      0
key                     95
mode                     0
danceability_%           0
valence_%                0
energy_%                 0
acousticness_%           0
instrumentalness_%       0
liveness_%               0
speechiness_%            0
released_season          0
LENGTH                   0
TOKENS                   0
POS                      0
ENTITIES                 0
dtype: int64

In [5]:
#Handling missing values
#We can fill "in_shazam_charts" column with the AVG of in_spotify_charts, in_apple_charts and in_deezer_charts

#mean_charts = sum(df[["in_spotify_charts", "in_apple_charts", "in_deezer_charts"]].mean())/3
##print(mean_charts)

# fill NaN values with the mean of each column
#df["in_shazam_charts"].fillna(mean_charts, inplace=True)

#We decide to drop the key and the shazam_charts columns because they both have more that 5% of total missing. And in the case of Shazam charts, Shazam was acquired by Apple in 2018, 
#so this variable is not going to add much different information than the Apple related variables.

df.drop("key", axis=1, inplace=True)
df.drop("in_shazam_charts", axis=1, inplace=True)

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 952 entries, 0 to 951
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   artist_count          952 non-null    int64 
 1   in_spotify_playlists  952 non-null    int64 
 2   in_spotify_charts     952 non-null    int64 
 3   streams               952 non-null    int64 
 4   in_apple_playlists    952 non-null    int64 
 5   in_apple_charts       952 non-null    int64 
 6   in_deezer_playlists   952 non-null    int64 
 7   in_deezer_charts      952 non-null    int64 
 8   bpm                   952 non-null    int64 
 9   mode                  952 non-null    object
 10  danceability_%        952 non-null    int64 
 11  valence_%             952 non-null    int64 
 12  energy_%              952 non-null    int64 
 13  acousticness_%        952 non-null    int64 
 14  instrumentalness_%    952 non-null    int64 
 15  liveness_%            952 non-null    in

In [248]:
df.isnull().sum()

artist_count            0
in_spotify_playlists    0
in_spotify_charts       0
streams                 0
in_apple_playlists      0
in_apple_charts         0
in_deezer_playlists     0
in_deezer_charts        0
bpm                     0
mode                    0
danceability_%          0
valence_%               0
energy_%                0
acousticness_%          0
instrumentalness_%      0
liveness_%              0
speechiness_%           0
released_season         0
LENGTH                  0
TOKENS                  0
POS                     0
ENTITIES                0
dtype: int64

In [174]:
df.shape

(952, 22)

In [16]:
# Check for duplicates
#df.duplicated().sum()

In [6]:
print(df.corr(numeric_only=True)["streams"].sort_values(ascending=False))


streams                 1.000000
in_spotify_playlists    0.789822
in_apple_playlists      0.772063
in_deezer_playlists     0.598131
in_apple_charts         0.320234
in_spotify_charts       0.245821
in_deezer_charts        0.228598
bpm                    -0.002438
acousticness_%         -0.004485
energy_%               -0.026051
valence_%              -0.040831
instrumentalness_%     -0.044902
liveness_%             -0.048337
danceability_%         -0.105457
speechiness_%          -0.112333
artist_count           -0.136463
Name: streams, dtype: float64


In [7]:
# dropping low correlated variables (<0.1)

df.drop(columns= ["bpm", "acousticness_%", "energy_%", "valence_%", "instrumentalness_%", "liveness_%"], axis=1, inplace=True)

In [8]:
df.head()

,artist_count,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,in_apple_charts,in_deezer_playlists,in_deezer_charts,mode,danceability_%,speechiness_%,released_season
0,2,553,147,141381703,43,263,45,10,Major,80,4,Summer
1,1,1474,48,133716286,48,126,58,14,Major,71,4,Winter
2,1,1397,113,140003974,94,207,91,14,Major,51,6,Summer
3,1,7858,100,800840817,116,207,125,12,Major,55,15,Summer
4,1,3133,50,303236322,84,133,87,15,Minor,65,6,Spring


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 952 entries, 0 to 951
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   artist_count          952 non-null    int64 
 1   in_spotify_playlists  952 non-null    int64 
 2   in_spotify_charts     952 non-null    int64 
 3   streams               952 non-null    int64 
 4   in_apple_playlists    952 non-null    int64 
 5   in_apple_charts       952 non-null    int64 
 6   in_deezer_playlists   952 non-null    int64 
 7   in_deezer_charts      952 non-null    int64 
 8   mode                  952 non-null    object
 9   danceability_%        952 non-null    int64 
 10  speechiness_%         952 non-null    int64 
 11  released_season       952 non-null    object
dtypes: int64(10), object(2)
memory usage: 89.4+ KB


In [10]:
#Scale Numerical Data with StandardScaler()

from sklearn.preprocessing import StandardScaler

# Copying original dataframe
df_z = df.copy()

scaler = StandardScaler()
num_cols = df_z.select_dtypes(include=np.number).columns

df_z[num_cols] = scaler.fit_transform(df[num_cols])

df_z.head()

,artist_count,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,in_apple_charts,in_deezer_playlists,in_deezer_charts,mode,danceability_%,speechiness_%,released_season
0,0.496468,-0.588758,6.896440,-0.657929,-0.287724,4.170501,-0.301230,1.214732,Major,0.890052,-0.619429,Summer
1,-0.623526,-0.472135,1.838224,-0.671459,-0.229871,1.463112,-0.289730,1.877534,Major,0.274608,-0.619429,Winter
2,-0.623526,-0.481885,5.159275,-0.660361,0.302382,3.063831,-0.260539,1.877534,Major,-1.093045,-0.417616,Summer
3,-0.623526,0.336248,4.495065,0.506043,0.556937,3.063831,-0.230464,1.546133,Major,-0.819514,0.490540,Summer
4,-0.623526,-0.262062,1.940410,-0.372249,0.186675,1.601446,-0.264078,2.043235,Minor,-0.135688,-0.417616,Spring


# XGBOOST Machine Learning Model
including only variables with more than 0.1 correlation

##  Specifying Feature and Target variables for our model and spliting the sample into training and testing sets

In [11]:
# Select Features
X = df_z.drop('streams', axis=1)

# Select Target
y = df_z['streams']


In [12]:
X.dtypes

artist_count            float64
in_spotify_playlists    float64
in_spotify_charts       float64
in_apple_playlists      float64
in_apple_charts         float64
in_deezer_playlists     float64
in_deezer_charts        float64
mode                     object
danceability_%          float64
speechiness_%           float64
released_season          object
dtype: object

In [13]:

#We have 2 categorical variables. Normally, you would encode them with ordinal or one-hot encoding, but XGBoost has the ability to internally deal with categoricals.
#The way to enable this feature is to cast the categorical columns into Pandas category data type (by default, they are treated as text columns)

# Extract text features
cats = X.select_dtypes(exclude=np.number).columns.tolist()

# Convert to Pandas category
for col in cats:
   X[col] = X[col].astype('category')



In [14]:
X.dtypes


artist_count             float64
in_spotify_playlists     float64
in_spotify_charts        float64
in_apple_playlists       float64
in_apple_charts          float64
in_deezer_playlists      float64
in_deezer_charts         float64
mode                    category
danceability_%           float64
speechiness_%            float64
released_season         category
dtype: object

In [15]:
#Split the data

# Set Training and Testing Data

X_train, X_test, y_train, y_test = train_test_split(X , y, 
                                                    shuffle = True, 
                                                    test_size=0.2, 
                                                    random_state=42)

# Show the Training and Testing Data
print('Shape of training feature:', X_train.shape)
print('Shape of testing feature:', X_test.shape)
print('Shape of training label:', y_train.shape)
print('Shape of testing label:', y_test.shape)

Shape of training feature: (761, 11)
Shape of testing feature: (191, 11)
Shape of training label: (761,)
Shape of testing label: (191,)


In [16]:
#Now, the important part: XGBoost comes with its own class for storing datasets called DMatrix. 
#It is a highly optimized class for memory and speed. That's why converting datasets into this format is a requirement for the native XGBoost API
#To enable automatic encoding of Pandas category columns, we also set enable_categorical to True

import xgboost as xgb

# Create regression matrices
dtrain_reg = xgb.DMatrix(X_train, y_train, enable_categorical=True)
dtest_reg = xgb.DMatrix(X_test, y_test, enable_categorical=True)

In [17]:
# Training

In [18]:
# Define hyperparameters
params = {"objective": "reg:squarederror", "tree_method": "hist"}

In [19]:
n = 100
model = xgb.train(
   params=params,
   dtrain=dtrain_reg,
   num_boost_round=n,
)

In [20]:
#Evaluation
#During the boosting rounds, the model object has learned all the patterns of the training set it possibly can. Now, we must measure its performance by testing it on unseen data

from sklearn.metrics import mean_squared_error

preds = model.predict(dtest_reg)


In [21]:
#mse = np.mean((actual - predicted) ** 2)
#rmse = np.sqrt(mse)

rmse = mean_squared_error(y_test, preds, squared=False)

print(f"RMSE of the base model: {rmse:.3f}")

#There are two ways we can improve it— by performing cross-validation and hyperparameter tuning. 

RMSE of the base model: 0.409


In [22]:
# Another approach to train the model is using validation sets. 
#We will use evaluation arrays that allow us to see model performance as it gets improved incrementally across boosting rounds

params = {"objective": "reg:squarederror", "tree_method": "hist"}
n = 100

In [23]:
evals = [(dtrain_reg, "train"), (dtest_reg, "validation")]

In [24]:
#When we pass this array to the evals parameter of xgb.train, we will see the model performance after each boosting round

evals = [(dtrain_reg, "train"), (dtest_reg, "validation")]

model = xgb.train(
   params=params,
   dtrain=dtrain_reg,
   num_boost_round=n,
   evals=evals,
)

[0]	train-rmse:0.77840	validation-rmse:0.67252
[1]	train-rmse:0.60290	validation-rmse:0.55260
[2]	train-rmse:0.47429	validation-rmse:0.47091
[3]	train-rmse:0.38508	validation-rmse:0.43485
[4]	train-rmse:0.32428	validation-rmse:0.41632
[5]	train-rmse:0.27712	validation-rmse:0.41014
[6]	train-rmse:0.24368	validation-rmse:0.40439
[7]	train-rmse:0.22223	validation-rmse:0.40219
[8]	train-rmse:0.20571	validation-rmse:0.40551
[9]	train-rmse:0.18943	validation-rmse:0.40140
[10]	train-rmse:0.17792	validation-rmse:0.40509
[11]	train-rmse:0.16675	validation-rmse:0.40876
[12]	train-rmse:0.15922	validation-rmse:0.40839
[13]	train-rmse:0.15235	validation-rmse:0.41048
[14]	train-rmse:0.14284	validation-rmse:0.41032
[15]	train-rmse:0.14103	validation-rmse:0.41046
[16]	train-rmse:0.13682	validation-rmse:0.41258
[17]	train-rmse:0.13424	validation-rmse:0.41491
[18]	train-rmse:0.12752	validation-rmse:0.41392
[19]	train-rmse:0.12346	validation-rmse:0.41126
[20]	train-rmse:0.12177	validation-rmse:0.41054
[2

In [25]:
#the model minimizes the score from  train-rmse:0.77 to validation-rmse:0.40

In [26]:
# using the verbose_eval parameter to reduce the number of rows in the output

params = {"objective": "reg:squarederror", "tree_method": "hist"}
n = 100

evals = [(dtest_reg, "validation"), (dtrain_reg, "train")]


model = xgb.train(
   params=params,
   dtrain=dtrain_reg,
   num_boost_round=n,
   evals=evals,
   verbose_eval=10 # Every ten rounds
)

[0]	validation-rmse:0.67252	train-rmse:0.77840
[10]	validation-rmse:0.40509	train-rmse:0.17792
[20]	validation-rmse:0.41054	train-rmse:0.12177
[30]	validation-rmse:0.41110	train-rmse:0.08565
[40]	validation-rmse:0.40777	train-rmse:0.05992
[50]	validation-rmse:0.40968	train-rmse:0.04318
[60]	validation-rmse:0.40905	train-rmse:0.03147
[70]	validation-rmse:0.40954	train-rmse:0.02420
[80]	validation-rmse:0.40918	train-rmse:0.01795
[90]	validation-rmse:0.40938	train-rmse:0.01258
[99]	validation-rmse:0.40950	train-rmse:0.01009


In [27]:
#Early stopping. Boosting rounds are important, as the XGboost model tries to minimize the loss in each round, however the loss will not allways go down,
#sometimes too many rounds can lead to overfitting
#a model that learned just enough patterns in training that it gives the highest performance on the validation set. 
#So, how do we find the perfect number of boosting rounds, then? Using Early stopping
#Early stopping forces XGBoost to watch the validation loss, and if it stops improving for a specified number of rounds, it automatically stops training.



In [28]:
n = 10000


model = xgb.train(
   params=params,
   dtrain=dtrain_reg,
   num_boost_round=n,
   evals=evals,
   verbose_eval=50,
   # Activate early stopping
   early_stopping_rounds=50
)

[0]	validation-rmse:0.67252	train-rmse:0.77840
[50]	validation-rmse:0.40968	train-rmse:0.04318
[100]	validation-rmse:0.40954	train-rmse:0.00976
[150]	validation-rmse:0.40927	train-rmse:0.00242
[200]	validation-rmse:0.40932	train-rmse:0.00169
[250]	validation-rmse:0.40932	train-rmse:0.00169
[300]	validation-rmse:0.40932	train-rmse:0.00169
[350]	validation-rmse:0.40932	train-rmse:0.00169
[400]	validation-rmse:0.40932	train-rmse:0.00169
[450]	validation-rmse:0.40932	train-rmse:0.00169
[500]	validation-rmse:0.40932	train-rmse:0.00169
[550]	validation-rmse:0.40932	train-rmse:0.00169
[600]	validation-rmse:0.40932	train-rmse:0.00169
[650]	validation-rmse:0.40932	train-rmse:0.00169
[700]	validation-rmse:0.40932	train-rmse:0.00169
[750]	validation-rmse:0.40932	train-rmse:0.00169
[800]	validation-rmse:0.40932	train-rmse:0.00169
[850]	validation-rmse:0.40932	train-rmse:0.00169
[900]	validation-rmse:0.40932	train-rmse:0.00169
[950]	validation-rmse:0.40932	train-rmse:0.00169
[1000]	validation-rmse:

In [29]:
#Model optimization

#XGBoost Cross-validation

#In cross-validation, we still have two sets: training and testing.
#After all folds are done, we can take the mean of the scores as the final, most realistic performance of the model.

params = {"objective": "reg:squarederror", "tree_method": "hist"}
n = 1000

results = xgb.cv(
   params, dtrain_reg,
   num_boost_round=n,
   nfold=5,
   early_stopping_rounds=20
)


In [30]:
results.head()

,train-rmse-mean,train-rmse-std,test-rmse-mean,test-rmse-std
0,0.775744,0.020410,0.808779,0.085744
1,0.599036,0.012583,0.680536,0.065949
2,0.473587,0.009504,0.608265,0.062489
3,0.384355,0.008010,0.569158,0.063538
4,0.319093,0.006265,0.545162,0.064845


In [31]:
# It has the same number of rows as the number of boosting rounds. Each row is the average of all splits for that round. 
#So, to find the best score, we take the minimum of the test-rmse-mean column

best_rmse = results['test-rmse-mean'].min()
best_rmse

0.5192356579995193